## File Encryption/Decryption using AES and RSA

This program utilizes the python hazmat cryptography library to encrypt and decrypt user files. The file will first be padded using PKCS7 to make sure the input to our encryption algorithm is the appropriate block size. CBC mode (AES) will then be used to encrypt a file of the user's choice. The encryption of a file is further enhanced with RSA encryption on the AES encryption key, and the program will also write the rsa public and private key to files. The output of the program will be a JSON object that contains the ciphertext, iv, RSA encrypted encryption key, and the file extension. The JSON object can then be read and then decrypted using the RSA private key to decrypt the encryption key and then using the encryption key to decrypt the ciphertext.

In [1]:
# import the necessary libraries needed
import os as os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding as asymPadding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from PIL import Image
import io
import json
import base64

In [2]:
# constants used for our encryption algorithm
IV_LENGTH = 16
KEY_LENGTH = 32
BLOCK_SIZE = 128

In [3]:
def myEncryptMAC(message):
    """
    Encrypts the message using CBC mode (AES).
    
    Arguments:
    message -- stream of bytes the user wants to encrypt
    
    Return:
    cipher -- The cipher text
    iv -- the IV needed for decryption
    key -- the key needed for decryption
    """
    
    # generate our key and iv
    backend = default_backend()
    iv = os.urandom(IV_LENGTH)
    encryption_key = os.urandom(KEY_LENGTH)
    integrity_key = os.urandom(KEY_LENGTH)
    
    # pad to approprate block size
    padder = padding.PKCS7(BLOCK_SIZE).padder()
    paddedMessage = padder.update(message)
    paddedMessage += padder.finalize()
    
    # encrypt using CBC mode (AES)
    cipher = Cipher(algorithms.AES(encryption_key), modes.CBC(iv), backend=backend)
    encryptor = cipher.encryptor()
    ct = encryptor.update(paddedMessage) + encryptor.finalize()
    
    # create tag of cipher using HMAC
    tag = hmac.HMAC(integrity_key, hashes.SHA256(), backend=default_backend())
    tag.update(ct)
    tag = tag.finalize()
    
    return ct, iv, tag, encryption_key, integrity_key

In [4]:
def myFileEncryptMAC(filepath):
    """
    Takes the file from the filepath specified by the user and encrypts the file. 
    
    Arguments:
    filepath -- Filepath for the file user wants to encrypt
    
    Return:
    cipher -- The cipher text
    iv -- the IV needed for decryption
    key -- the key needed for decryption
    fileType -- the file extension
    """
    # read the file and convert it to bytes
    file = open(filepath, "rb")
    byte = bytes(file.read())
    file.close()
    
    # encrypt the file
    cipher, iv, tag, encryption_key, integrity_key = myEncryptMAC(byte)
    extIndex = filepath.find('.')
    fileType = filepath[extIndex+1:]
    return cipher, iv, tag, encryption_key, integrity_key, fileType

In [5]:
def myRSAEncrypt(filepath):
    """
    Takes the file from the filepath specified by the user and encrypts the file and also the encryption key. 
    
    Arguments:
    filepath -- Filepath for the file user wants to encrypt
    
    Return:
    cipher -- The cipher text
    iv -- the IV needed for decryption
    RSAencryptedKey -- the RSA encrypted encryption key needed for decryption
    fileType -- the file extension
    """
    # Generate RSA private key and public key
    private_key = rsa.generate_private_key(
        public_exponent = 65537,
        key_size = 2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()

    # Write out RSA private key and public key to file using serialization
    RSAprivateKey = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    )
    RSApublicKey = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo,
    )
    
    # Output the private key
    rsaFile = open("RSAprivateKey.pem", "wb")
    rsaFile.write(RSAprivateKey)
    rsaFile.close()
    
    # Output the public key
    rsaFile = open("RSApublicKey.pem", "wb")
    rsaFile.write(RSApublicKey)
    rsaFile.close()

    # Encrypt our file of choice using myFileEncrypt()
    cipher, iv, tag, encryption_key, integrity_key, fileType = myFileEncryptMAC(filepath)
    
    # concatenate encryption key and integrity key
    enc_int_key_concat = encryption_key + integrity_key
    
    # Encrypt the encryption key using the RSA public key with OAEP
    RSAencryptedCipher = public_key.encrypt(
        enc_int_key_concat,
        asymPadding.OAEP(
            mgf=asymPadding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    return cipher, iv, tag, RSAencryptedCipher, fileType

In [6]:
def myDecrypt(ciphertext, key, iv):
    """
    Decrypts the message using CBC mode (AES).
    
    Arguments:
    ciphertext -- The cipher text
    iv -- the IV needed for decryption
    key -- the key needed for decryption
    
    Return:
    unpaddedMessage -- the original message
    """
    # decrypt the file
    backend = default_backend()
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=backend)
    decryptor = cipher.decryptor()
    decryptedMessage = decryptor.update(ciphertext) + decryptor.finalize()
    
    # unpad the file
    unpadder = padding.PKCS7(BLOCK_SIZE).unpadder()
    unpaddedMessage = unpadder.update(decryptedMessage)
    
    return unpaddedMessage

In [7]:
def myFileDecrypt(ciphertext, iv, key, fileType):
    """
    Takes the file from the filepath specified by the user and decrypts the file. 
    
    Arguments:
    filepath -- The pathing for the file the user wants to decrypt
    iv -- the IV needed for decryption
    key -- the key needed for decryption
    fileType -- the file extension
    """
    decryptedFile = myDecrypt(ciphertext, key, iv)
    image = Image.open(io.BytesIO(decryptedFile))
    image.show()

In [8]:
def myRSADecrypt(ciphertext, rsaCipher, iv,tag, fileType, rsaPrivateKeyPath):
    """
    Decrypts the ciphertext by first decrypting the encryption key cipher using the RSA private key,
    then using the decrypted encryption key to decrypt the ciphertext.
    
    Arguments:
    cipher -- The ciphertext
    rsaCipher -- The ciphertext for the encryption key
    iv -- the IV needed for decryption
    tag -- the tag generated by MAC when hashing the cyphertext
    fileType -- the extension for the type of the file being decrypted
    rsaPrivateKeyPath -- the file pathing for the RSA private key
    
    Return:
    message -- the original message
    """
    
    # read in the RSA private key
    key_file = open(rsaPrivateKeyPath, "rb")
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
        backend=default_backend()
    )
    
    # Decrypt the hash of the concatenated keys using the private key
    concatKey = private_key.decrypt(
        rsaCipher,
        asymPadding.OAEP(
            mgf=asymPadding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    # get the encryption key from concatenated key
    encryption_key = concatKey[:32]
    
    # get the integrity key from concatenated key
    integrity_key = concatKey[32:]

    # MAC Verification
    verify = hmac.HMAC(integrity_key, hashes.SHA256(), backend=default_backend())
    verify.update(ciphertext)
    verify.verify(tag)
    
    myFileDecrypt(ciphertext, iv, encryption_key,fileType)

In [9]:
# Encrypt a file of the user's choice
cipher, iv, tag, RSAencryptedCipher, fileType = myRSAEncrypt("test.png")

files = {
    "cipher" : str(cipher),
    "iv" : str(iv),
    "tag": str(tag),
    "key": str(RSAencryptedCipher),
    "ext"  : fileType   
}

fileName = open("encrypt.json", "w")
jsonObject = json.dump(files,fileName)
fileName.close()

png


In [10]:
# Decrypt the file of the user's choice
fileName = open("encrypt.json", "r")
jsonObject = json.load(fileName)
cipherTest = jsonObject["cipher"]
ivTest = jsonObject["iv"]
RSAencryptedKeyTest = jsonObject["key"]
fileTypeTest = jsonObject["ext"]
fileName.close()

myRSADecrypt(cipher,RSAencryptedCipher, iv, tag, fileType, "RSAprivateKey.pem")

C:\Users\piercingonez\Anaconda3\lib\site-packages\PIL\Image.py:888: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [11]:
def encryptEverythingInDirectory():
    currentDirectory = os.getcwd()
    filesList = os.listdir(currentDirectory)

    for x in filesList:
        if (not x.endswith('.ipynb') and not x.endswith('.pem') and not x.endswith('.json') and not x.endswith('.bytes')):
            myRSAEncrypt(x)